# Phase 1

## Preprocessing

In [1]:
import warnings

import pandas as pd
from dateutil.parser import parse
from tmdbv3api import Movie
from tmdbv3api import TMDb

import keys  # TMDb API key file

warnings.filterwarnings("ignore")

##### Reading the train datasets

In [ ]:
# reading csv files
movies_base = pd.read_csv('datasets/1/train/movies-revenue.csv')
movies_director = pd.read_csv('datasets/1/train/movie-director.csv')

##### Checking nulls

In [ ]:
movies_base.isnull().sum()

##### Joining directors dataset into the movies (main) dataset

In [ ]:
# Merging dataframes; how = 'left' := use only keys from left frame.
df = pd.merge(movies_base, movies_director, on='name', how='left')

# displaying result
df.head()

##### Filling the null directors

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:
df.dtypes

In [ ]:
# Formatting the release_date to match the TMDb date format

for i, movie in df.iterrows():
    # Parse date from a string and return a datetime.datetime
    release_date = parse(movie['release_date'])
    # Remove the time from it reducing it to just the date
    release_date = release_date.date()

    # Parser doesn't do well with dates prior to the 80s
    # Correcting the dates newer than this year to a century earlier
    if release_date.year > 2023:
        release_date = release_date.replace(year=release_date.year - 100)

    # Editing the value at the original dataframe
    df.at[i, 'release_date'] = release_date

df['release_date'] = pd.to_datetime(df['release_date'])
df.head(n=10)

##### Initializing TMDb API

In [ ]:
# Using the TMDb to fill out the missing director from the original dataset

# https://github.com/AnthonyBloomer/tmdbv3api
# https://developers.themoviedb.org/3/getting-started/introduction

# Creating a base class instance from the api library
tmdb = TMDb()
tmdb.api_key = keys.tmdb_key
tmdb.language = 'en'
tmdb.debug = True


##### Using the TMDb API to fill missing directors via the movie title and its release date
Since the director name is still not encoded, the director's popularity score will replace it.

In [ ]:
# Creating a Movie instance to search by the movie details
movie = Movie()

for i, mov in df.iterrows():
    search = movie.search(mov['name'])  # Search by the movie title
    for res in search:
        try:
            # Confirming the search results by the release date year
            mov_date_str = str(mov['release_date'])
            if res['release_date'][:4] == mov_date_str[:4]:
                # Extracting the director from the movie credits
                for member in movie.credits(res.id)['crew']:
                    if member['job'] == 'Director':
                        # Editing the value at the original dataframe
                        df.at[i, 'director'] = member['popularity']
                        break
                break
        except BaseException as error:
            print('An exception occurred: {}'.format(error) + " " + mov['name'])

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

##### Cleaning up the revenue to extract the float value

In [ ]:
df['revenue'] = df['revenue'].str.replace(',', '').str.replace('$', '').astype('float')
print(df.dtypes)
df.head()

##### Encoding the MPAA Rating and Genre to indicator variables

In [ ]:
# Fill the nulls with the mode
df['director'] = df['director'].fillna(value=df['director'].median())
df['genre'] = df['genre'].fillna(value=df['genre'].mode()[0])
df['MPAA_rating'] = df['MPAA_rating'].fillna(value=df['MPAA_rating'].mode()[0])
df.isnull().sum()

In [ ]:
# Preprocessing genre and MPAA_rating

df = pd.get_dummies(df, columns=["MPAA_rating"], prefix=["rating_is"])
df = pd.get_dummies(df, columns=["genre"], prefix=["genre_is"])
df.head()

##### Encoding the release date to a scalar

In [ ]:
def date_to_float(dt):
    # Calculating the months and days
    calc = (((dt.month - 1) * 30) + dt.day) / 365
    # Adding calc to the years
    return dt.year + calc

##### Splitting date to days and months and then converting it to a scalar via `date_to_float()`

In [ ]:
df['month'] = df['release_date'].dt.month
df['day'] = df['release_date'].dt.day
df['season'] = df['release_date'].dt.quarter
df['release_date'] = df['release_date'].apply(date_to_float)
df.head()

##### Saving the preprocessed dataset

In [ ]:
df.to_csv('datasets/1/train/preprocessed.csv', index=False)

##### Adjusting the revenue to inflation

In [2]:
import cpi

cpi.update()
df = pd.read_csv('datasets/1/train/preprocessed.csv')

# Adjust the revenue to inflation using cpi
df['revenue'] = df.apply(lambda x: cpi.inflate(x['revenue'],
                                               int(x.release_date)), axis=1)

##### Feature Engineering

In [ ]:
df_animation = pd.read_csv('datasets/1/train/movie-voice-actors.csv')

# Remove duplicates on name column
df_animation = df_animation.drop_duplicates(subset=['name'], keep='first')

# Add column is_animation
df_animation['is_animation'] = 1

# Drop other columns that are not needed
df_animation = df_animation.drop(['voice_actor', 'character'], axis=1)

df_animation.head()

##### Merging the animation dataset with the main dataset

In [ ]:
# Merge the animation dataset with the main dataset
df = pd.merge(df, df_animation, on='name', how='left')

# Fill the nulls with 0
df['is_animation'] = df['is_animation'].fillna(value=0)

df.head()

##### Fitting the Linear Regression model

In [3]:
from scipy.stats import stats
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df = df.drop(['name'], axis=1)

# drop outliers from the dataset via z score
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

X = df.drop(['revenue'], axis=1)
Y = df['revenue']

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Feature Scaling
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting Linear Regression to the dataset
regressor = linear_model.LinearRegression()
regressor.fit(X_train, Y_train)

# Predicting the Test set results
Y_pred = regressor.predict(X_test)

# The coefficients
print('Coefficients: \n', regressor.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % metrics.mean_squared_error(Y_test, Y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % metrics.r2_score(Y_test, Y_pred))


Coefficients: 
 [-1.91251920e+08  9.98980854e+07  4.50903914e+07  2.23517418e-08
  4.77233973e+06 -6.43895890e+06 -4.34237722e+07  5.96046448e-08
  6.05348541e+07  3.72529030e-08 -8.56900466e+06  0.00000000e+00
  0.00000000e+00 -5.19658495e+07  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  3.24374329e+08
  1.66396961e+07 -1.81530738e+08]
Mean squared error: 48311858652198056.00
Coefficient of determination: 0.24
